In [1]:
import pandas as pd

In [2]:
import regex as re

In [3]:
from itertools import chain
from collections import Counter

In [4]:
path = r"quran-simple-clean.txt"

In [5]:
with open(path, encoding='utf-8') as f:
    quran = [line.strip() for line in f.readlines()]
    quran = [line for line in quran if line]

In [6]:
df = pd.DataFrame(data={'Q': quran})

In [7]:
df['Q'] = df.Q.apply(lambda x: x.split('|'))

In [8]:
df

,Q
0,"[1, 1, بسم الله الرحمن الرحيم]"
1,"[1, 2, الحمد لله رب العالمين]"
2,"[1, 3, الرحمن الرحيم]"
3,"[1, 4, مالك يوم الدين]"
4,"[1, 5, إياك نعبد وإياك نستعين]"
...,...
6231,"[114, 2, ملك الناس]"
6232,"[114, 3, إله الناس]"
6233,"[114, 4, من شر الوسواس الخناس]"
6234,"[114, 5, الذي يوسوس في صدور الناس]"


In [9]:
df['SURA_NO'] = df.Q.apply(lambda x: x[0])

In [10]:
df['AYA_NO'] = df.Q.apply(lambda x: x[1])

In [11]:
df['AYA'] = df.Q.apply(lambda x: x[2])

In [12]:
set_chars = set(chain(*df['AYA']))

In [13]:
set_chars = sorted(list(set_chars))

In [14]:
for i in set_chars[-7:]:
    print(i)

ۖ
ۗ
ۘ
ۙ
ۚ
ۛ
ۜ


In [15]:
df = df.drop(columns=['Q'])

In [16]:
df['LEN_AYA'] = df['AYA'].apply(lambda x: len(x.split()))

In [17]:
dict_sura_name = pd.read_excel('../DATASETS/quran_sura_names_numbers.xlsx')

In [18]:
dict_sura_name = dict(zip(dict_sura_name.SURA_NO, dict_sura_name.AR_NAME))

In [19]:
df['SURA_NAME'] = df.SURA_NO.apply(lambda x:  dict_sura_name[int(x)])

In [20]:
with open('../DATASETS/Quran_Aya_Per_Line.txt', encoding='utf-8') as f:
    ar_clean_char = set(f.read())
    ar_clean_char = [i.strip() for i in ar_clean_char]
    ar_clean_char = [i for i in ar_clean_char if i]
    ar_clean_char = sorted(ar_clean_char)

In [21]:
pat_ar_clean_char = ''.join(ar_clean_char)

In [22]:
df.SURA_NAME = df.SURA_NAME.apply(lambda x: re.sub(f"[^{pat_ar_clean_char}]", '', x).strip())

In [23]:
df['SURA_NAME'] = df['SURA_NAME'].apply(lambda x: x[3:] if x[:3]== 'سور' else x)

In [24]:
df

,SURA_NO,AYA_NO,AYA,LEN_AYA,SURA_NAME
0,1,1,بسم الله الرحمن الرحيم,4,الفاتحة
1,1,2,الحمد لله رب العالمين,4,الفاتحة
2,1,3,الرحمن الرحيم,2,الفاتحة
3,1,4,مالك يوم الدين,3,الفاتحة
4,1,5,إياك نعبد وإياك نستعين,4,الفاتحة
...,...,...,...,...,...
6231,114,2,ملك الناس,2,الناس
6232,114,3,إله الناس,2,الناس
6233,114,4,من شر الوسواس الخناس,4,الناس
6234,114,5,الذي يوسوس في صدور الناس,5,الناس


In [25]:
#df.to_pickle('QURAN_AYA_WITH_PAUSES.pkl', protocol=4)

In [26]:
df

,SURA_NO,AYA_NO,AYA,LEN_AYA,SURA_NAME
0,1,1,بسم الله الرحمن الرحيم,4,الفاتحة
1,1,2,الحمد لله رب العالمين,4,الفاتحة
2,1,3,الرحمن الرحيم,2,الفاتحة
3,1,4,مالك يوم الدين,3,الفاتحة
4,1,5,إياك نعبد وإياك نستعين,4,الفاتحة
...,...,...,...,...,...
6231,114,2,ملك الناس,2,الناس
6232,114,3,إله الناس,2,الناس
6233,114,4,من شر الوسواس الخناس,4,الناس
6234,114,5,الذي يوسوس في صدور الناس,5,الناس


In [27]:
pauses = """
ۗ
ۘ
ۚ

""".strip().split('\n')

In [28]:
pauses = [pause.strip() for pause in pauses]

In [29]:
pauses

['ۗ', 'ۘ', 'ۚ']

In [30]:
def tokenize_if_long(aya , threshold=25):
    pauses = ['ۗ', 'ۘ', 'ۚ']
    special_regex = f"""[{''.join(pauses)}]+""".strip()
        
    aya_len = len(aya.split())
    
    if aya_len < threshold:
        return [aya.strip()]
    else:
        aya_tokenized = re.split(special_regex, aya)
        return [i.strip() for i in aya_tokenized]

In [31]:
df.AYA_NO = df.AYA_NO.astype(int)
df.SURA_NO = df.SURA_NO.astype(int)

In [32]:
def remove_basmallah(dframe):
    aya_no = dframe.AYA_NO
    aya_text = dframe.AYA
    if (aya_no == 1) and (aya_text.startswith('بسم الله الرحمن الرحيم'.strip())):
        return aya_text[22:].strip()
    else:
        return aya_text

In [34]:
df.AYA = df.apply(lambda x: remove_basmallah(x), axis=1)

In [36]:
df.LEN_AYA = df.AYA.apply(lambda x: len(x.split()))

In [39]:
df = df[df.LEN_AYA > 0].reset_index(drop=True)

In [40]:
df['AYA_TOKENIZED'] = df['AYA'].apply(lambda x: tokenize_if_long(x))

In [41]:
df.sample(30)
test = df.AYA_TOKENIZED.iloc[1737]

In [42]:
test

['ولقد استهزئ برسل من قبلك فأمليت للذين كفروا ثم أخذتهم ۖ فكيف كان عقاب']

In [43]:
[((idx + 1), aya) for (idx, aya) in  enumerate(test)]

[(1, 'ولقد استهزئ برسل من قبلك فأمليت للذين كفروا ثم أخذتهم ۖ فكيف كان عقاب')]

In [44]:
df['AYA_TOKENIZED_SERIAL'] = df.AYA_TOKENIZED.apply(lambda x: [(str(idx + 1), aya) for (idx, aya) in  enumerate(x)])

In [45]:
df

,SURA_NO,AYA_NO,AYA,LEN_AYA,SURA_NAME,AYA_TOKENIZED,AYA_TOKENIZED_SERIAL
0,1,2,الحمد لله رب العالمين,4,الفاتحة,[الحمد لله رب العالمين],"[(1, الحمد لله رب العالمين)]"
1,1,3,الرحمن الرحيم,2,الفاتحة,[الرحمن الرحيم],"[(1, الرحمن الرحيم)]"
2,1,4,مالك يوم الدين,3,الفاتحة,[مالك يوم الدين],"[(1, مالك يوم الدين)]"
3,1,5,إياك نعبد وإياك نستعين,4,الفاتحة,[إياك نعبد وإياك نستعين],"[(1, إياك نعبد وإياك نستعين)]"
4,1,6,اهدنا الصراط المستقيم,3,الفاتحة,[اهدنا الصراط المستقيم],"[(1, اهدنا الصراط المستقيم)]"
...,...,...,...,...,...,...,...
6230,114,2,ملك الناس,2,الناس,[ملك الناس],"[(1, ملك الناس)]"
6231,114,3,إله الناس,2,الناس,[إله الناس],"[(1, إله الناس)]"
6232,114,4,من شر الوسواس الخناس,4,الناس,[من شر الوسواس الخناس],"[(1, من شر الوسواس الخناس)]"
6233,114,5,الذي يوسوس في صدور الناس,5,الناس,[الذي يوسوس في صدور الناس],"[(1, الذي يوسوس في صدور الناس)]"


In [46]:
df = df.explode('AYA_TOKENIZED_SERIAL').reset_index(drop=True)

In [47]:
df

,SURA_NO,AYA_NO,AYA,LEN_AYA,SURA_NAME,AYA_TOKENIZED,AYA_TOKENIZED_SERIAL
0,1,2,الحمد لله رب العالمين,4,الفاتحة,[الحمد لله رب العالمين],"(1, الحمد لله رب العالمين)"
1,1,3,الرحمن الرحيم,2,الفاتحة,[الرحمن الرحيم],"(1, الرحمن الرحيم)"
2,1,4,مالك يوم الدين,3,الفاتحة,[مالك يوم الدين],"(1, مالك يوم الدين)"
3,1,5,إياك نعبد وإياك نستعين,4,الفاتحة,[إياك نعبد وإياك نستعين],"(1, إياك نعبد وإياك نستعين)"
4,1,6,اهدنا الصراط المستقيم,3,الفاتحة,[اهدنا الصراط المستقيم],"(1, اهدنا الصراط المستقيم)"
...,...,...,...,...,...,...,...
7436,114,2,ملك الناس,2,الناس,[ملك الناس],"(1, ملك الناس)"
7437,114,3,إله الناس,2,الناس,[إله الناس],"(1, إله الناس)"
7438,114,4,من شر الوسواس الخناس,4,الناس,[من شر الوسواس الخناس],"(1, من شر الوسواس الخناس)"
7439,114,5,الذي يوسوس في صدور الناس,5,الناس,[الذي يوسوس في صدور الناس],"(1, الذي يوسوس في صدور الناس)"


In [48]:
df['UNIQUE_TOKEN'] = df.apply(lambda x: f"""
{str(x["SURA_NO"]).zfill(3)}_{str(x["AYA_NO"]).zfill(3)}_{str(x['AYA_TOKENIZED_SERIAL'][0]).zfill(3)}
""".strip(), axis=1)

In [49]:
df

,SURA_NO,AYA_NO,AYA,LEN_AYA,SURA_NAME,AYA_TOKENIZED,AYA_TOKENIZED_SERIAL,UNIQUE_TOKEN
0,1,2,الحمد لله رب العالمين,4,الفاتحة,[الحمد لله رب العالمين],"(1, الحمد لله رب العالمين)",001_002_001
1,1,3,الرحمن الرحيم,2,الفاتحة,[الرحمن الرحيم],"(1, الرحمن الرحيم)",001_003_001
2,1,4,مالك يوم الدين,3,الفاتحة,[مالك يوم الدين],"(1, مالك يوم الدين)",001_004_001
3,1,5,إياك نعبد وإياك نستعين,4,الفاتحة,[إياك نعبد وإياك نستعين],"(1, إياك نعبد وإياك نستعين)",001_005_001
4,1,6,اهدنا الصراط المستقيم,3,الفاتحة,[اهدنا الصراط المستقيم],"(1, اهدنا الصراط المستقيم)",001_006_001
...,...,...,...,...,...,...,...,...
7436,114,2,ملك الناس,2,الناس,[ملك الناس],"(1, ملك الناس)",114_002_001
7437,114,3,إله الناس,2,الناس,[إله الناس],"(1, إله الناس)",114_003_001
7438,114,4,من شر الوسواس الخناس,4,الناس,[من شر الوسواس الخناس],"(1, من شر الوسواس الخناس)",114_004_001
7439,114,5,الذي يوسوس في صدور الناس,5,الناس,[الذي يوسوس في صدور الناس],"(1, الذي يوسوس في صدور الناس)",114_005_001


In [50]:
df['AYA_SENTENCE'] = df['AYA_TOKENIZED_SERIAL'].apply(lambda x: x[1].strip())

In [51]:
df['LEN_AYA_SENTENCE'] = df['AYA_SENTENCE'].apply(lambda x: len(x.split()))

In [52]:
df['LEN_AYA_SENTENCE'].median(), df['LEN_AYA_SENTENCE'].max(), 

(9.0, 73)

In [53]:
df[(df.LEN_AYA_SENTENCE>60) & (df.LEN_AYA_SENTENCE<100)]

,SURA_NO,AYA_NO,AYA,LEN_AYA,SURA_NAME,AYA_TOKENIZED,AYA_TOKENIZED_SERIAL,UNIQUE_TOKEN,AYA_SENTENCE,LEN_AYA_SENTENCE
445,2,259,أو كالذي مر على قرية وهي خاوية على عروشها قال ...,74,البقرة,[أو كالذي مر على قرية وهي خاوية على عروشها قال...,"(1, أو كالذي مر على قرية وهي خاوية على عروشها ...",002_259_001,أو كالذي مر على قرية وهي خاوية على عروشها قال ...,62
1280,5,110,إذ قال الله يا عيسى ابن مريم اذكر نعمتي عليك و...,69,المائدة,[إذ قال الله يا عيسى ابن مريم اذكر نعمتي عليك ...,"(1, إذ قال الله يا عيسى ابن مريم اذكر نعمتي عل...",005_110_001,إذ قال الله يا عيسى ابن مريم اذكر نعمتي عليك و...,69
3737,24,31,وقل للمؤمنات يغضضن من أبصارهن ويحفظن فروجهن ول...,82,النور,[وقل للمؤمنات يغضضن من أبصارهن ويحفظن فروجهن و...,"(1, وقل للمؤمنات يغضضن من أبصارهن ويحفظن فروجه...",024_031_001,وقل للمؤمنات يغضضن من أبصارهن ويحفظن فروجهن ول...,73


In [54]:
len(df['AYA'].iloc[30].split())

20

In [55]:
print(df['AYA'].iloc[3739])

وأنكحوا الأيامى منكم والصالحين من عبادكم وإمائكم ۚ إن يكونوا فقراء يغنهم الله من فضله ۗ والله واسع عليم


In [58]:
df[df['AYA'].apply(lambda x: 'حرمت عليكم أمه' in x)]

,SURA_NO,AYA_NO,AYA,LEN_AYA,SURA_NAME,AYA_TOKENIZED,AYA_TOKENIZED_SERIAL,UNIQUE_TOKEN,AYA_SENTENCE,LEN_AYA_SENTENCE
835,4,23,حرمت عليكم أمهاتكم وبناتكم وأخواتكم وعماتكم وخ...,55,النساء,[حرمت عليكم أمهاتكم وبناتكم وأخواتكم وعماتكم و...,"(1, حرمت عليكم أمهاتكم وبناتكم وأخواتكم وعماتك...",004_023_001,حرمت عليكم أمهاتكم وبناتكم وأخواتكم وعماتكم وخ...,49
836,4,23,حرمت عليكم أمهاتكم وبناتكم وأخواتكم وعماتكم وخ...,55,النساء,[حرمت عليكم أمهاتكم وبناتكم وأخواتكم وعماتكم و...,"(2, إن الله كان غفورا رحيما)",004_023_002,إن الله كان غفورا رحيما,5


In [59]:
len("""
وقل للمؤمنات يغضضن من أبصارهن ويحفظن فروجهن ولا يبدين زينتهن إلا ما ظهر منها ۖ وليضربن بخمرهن على جيوبهن ۖ ولا يبدين زينتهن إلا لبعولتهن أو آبائهن أو آباء بعولتهن أو أبنائهن أو أبناء بعولتهن أو إخوانهن أو بني إخوانهن أو بني أخواتهن أو نسائهن أو ما ملكت أيمانهن أو التابعين غير أولي الإربة من الرجال أو الطفل الذين لم يظهروا على عورات النساء ۖ ولا يضربن بأرجلهن ليعلم ما يخفين من زينتهن ۚ
""".strip().split())

74

In [60]:
test = df[(df.LEN_AYA_SENTENCE>=46)]

In [61]:
len(test)

11

In [ ]:
test = test.sort_values(['LEN_AYA_SENTENCE'], ascending=False).reset_index(drop=True)

In [ ]:
test['IDX'] = test.index

In [ ]:
test_idx = list(test.IDX)

In [ ]:
test_idx

In [ ]:
for i in test_idx:
    idx = test['IDX'].iloc[i]
    sent = test['AYA_SENTENCE'].iloc[i]
    len_sent = test['LEN_AYA_SENTENCE'].iloc[i]
    sura_name = test['SURA_NAME'].iloc[i]
    aya_no = test['AYA_NO'].iloc[i]
    
    print(
f"""
من سورة 👈: {sura_name} -- الآية: {aya_no} -- عدد الكلمات: {len_sent}
{sent}
{"-"*50}
""".strip())

In [ ]:
len(test_idx)

In [63]:
#df.to_pickle('QURAN_AYA_WITH_TOKENIZED_BY_PAUSES.pkl', protocol=4)

In [64]:
df

,SURA_NO,AYA_NO,AYA,LEN_AYA,SURA_NAME,AYA_TOKENIZED,AYA_TOKENIZED_SERIAL,UNIQUE_TOKEN,AYA_SENTENCE,LEN_AYA_SENTENCE
0,1,2,الحمد لله رب العالمين,4,الفاتحة,[الحمد لله رب العالمين],"(1, الحمد لله رب العالمين)",001_002_001,الحمد لله رب العالمين,4
1,1,3,الرحمن الرحيم,2,الفاتحة,[الرحمن الرحيم],"(1, الرحمن الرحيم)",001_003_001,الرحمن الرحيم,2
2,1,4,مالك يوم الدين,3,الفاتحة,[مالك يوم الدين],"(1, مالك يوم الدين)",001_004_001,مالك يوم الدين,3
3,1,5,إياك نعبد وإياك نستعين,4,الفاتحة,[إياك نعبد وإياك نستعين],"(1, إياك نعبد وإياك نستعين)",001_005_001,إياك نعبد وإياك نستعين,4
4,1,6,اهدنا الصراط المستقيم,3,الفاتحة,[اهدنا الصراط المستقيم],"(1, اهدنا الصراط المستقيم)",001_006_001,اهدنا الصراط المستقيم,3
...,...,...,...,...,...,...,...,...,...,...
7436,114,2,ملك الناس,2,الناس,[ملك الناس],"(1, ملك الناس)",114_002_001,ملك الناس,2
7437,114,3,إله الناس,2,الناس,[إله الناس],"(1, إله الناس)",114_003_001,إله الناس,2
7438,114,4,من شر الوسواس الخناس,4,الناس,[من شر الوسواس الخناس],"(1, من شر الوسواس الخناس)",114_004_001,من شر الوسواس الخناس,4
7439,114,5,الذي يوسوس في صدور الناس,5,الناس,[الذي يوسوس في صدور الناس],"(1, الذي يوسوس في صدور الناس)",114_005_001,الذي يوسوس في صدور الناس,5
